# PH output

In [1]:
pip install ..

Processing /home/jovyan
  Created wheel for aptviz: filename=aptviz-0.0.1-py3-none-any.whl size=4074 sha256=9b4753e5e084e90a7ed8fb5677d15d05884479923a39185ac210857609ae7a27
  Stored in directory: /tmp/pip-ephem-wheel-cache-u2_f9gl1/wheels/fc/c4/49/78b5bd16ca276f2916d0829d47c131046b6e4575f7dd51e987
Successfully built aptviz
  Attempting uninstall: aptviz
    Found existing installation: aptviz 0.0.1
    Uninstalling aptviz-0.0.1:
      Successfully uninstalled aptviz-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
## Load packages 

import numpy as np
import sys
import pandas as pd
import networkx as nx
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go

# sys.path.insert(1, "../aptviz")
from aptviz import aptviz_themes
from aptviz.supporting import *
import plotly.io as pio

# Load ExHACT visualization themes
pio.templates.default = "aptviz"
my_charcoal = "#3f4142"

In [3]:
## Generate fake data
n_nodes = 50
n_simps = [n_nodes, 100, 150, 100, 20, 5]
max_dim = len(n_simps)-1

fsc_df = create_fake_fsc_df(n_nodes, n_simps, max_dim)
fsc_df.head()

Created df with length 425. Expected 425.


,cell_id,dim,nodes,weight,faces,rank,is_maximal
0,0,0,[0],5.102835,[],29.0,1
1,1,0,[1],5.749967,[],44.0,1
2,2,0,[2],5.575383,[],14.0,0
3,3,0,[3],5.555838,[],6.0,1
4,4,0,[4],5.260795,[],1.0,1


In [4]:
## Generate fake barcode from fsc USING RANK

n_bars = 100
bar_df = create_fake_barcode(fsc_df, n_bars)
bar_df.head()

,bar_id,bar_dim,bar_birth,bar_death,rep,lifetime
0,0,3,104.0,281.0,"[86, 111, 89, 131]",177.0
1,1,2,107.0,225.0,"[123, 149, 104, 138]",118.0
2,2,0,95.0,270.0,"[52, 57, 63, 95]",175.0
3,3,2,85.0,198.0,"[143, 94, 103, 120]",113.0
4,4,0,69.0,172.0,"[56, 53, 130, 119]",103.0


In [6]:
## Plot overlayed persistence diagram

max_filtration = np.max(fsc_df["rank"])
fig = px.scatter(bar_df, x="bar_birth", y="bar_death", color="bar_dim",
                 title="Persistence Diagrams (overlayed)", hover_data = ["rep"],
                 category_orders = {"bar_dim":[str(x) for x in np.arange(max_dim)]})

fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
    range = (0,max_filtration),
    title_text="death"
  )

fig.update_xaxes(
    range = (0,max_filtration),
    constrain='domain',
    title_text="birth",
    layer = "above traces"
)

fig.update_traces(mode='markers', marker_opacity=0.7, marker_size=12, marker_line_color = my_charcoal,
                 marker_line_width = 0.4)

fig.add_trace(
    go.Scatter(
        x=[0, max_filtration],
        y=[0, max_filtration],
        mode="lines",
        line=go.scatter.Line(color= 'rgba(187, 190, 191, .9)', width=0.5),
        showlegend=False,
        opacity = 0.6)
)

fig.show()

# fig.write_image("fig1.pdf")



In [6]:

fig = px.scatter(bar_df, x="bar_birth", y="bar_death", color="bar_dim", template = "plotly_white",
                 title="Persistence Diagrams (faceted)", hover_data = ["rep"],
                 color_discrete_sequence=aptviz_themes.davos_colors,
                 facet_col = "bar_dim",
                 facet_col_wrap=3,
                 facet_col_spacing = 0.1,
                 facet_row_spacing = 0.3,
                 height = 300*(np.floor(max_dim/3)+1),
                 width = 800,
                 category_orders = {"bar_dim": [str(x) for x in np.arange(max_dim)]})

trace = go.Scatter(
        x=[0, max_filtration],
        y=[0, max_filtration],
        mode="lines",
        line=go.scatter.Line(color= 'rgba(187, 190, 191, .9)', width=0.5),
        showlegend=False)


# Add trace to all rows/cols, but not to empty subplots
fig.add_trace(trace, row="all", col="all", exclude_empty_subplots=True)


# Update axes
fig.update_yaxes(
    scaleratio = 1,
    range = (0,max_filtration),
    title_text="death"
  )

fig.update_xaxes(
    range = (0,max_filtration),
    constrain='domain',
    title_text="birth",
)


fig.show()

In [7]:
fig = go.Figure()
bar_df_sorted = bar_df.sort_values(by=["bar_dim", "bar_birth"])
bar_df_sorted = bar_df_sorted.reset_index(drop=True)

for b,bar in bar_df_sorted.iterrows():
    
    if (bar_df_sorted.iloc[b-1]["bar_dim"] != bar["bar_dim"]) :
        showlegendtf = True
    else :
        showlegendtf = False
        
    fig.add_trace(go.Scatter(x=[bar["bar_birth"], bar["bar_death"]], y=[b+1, b+1],
                    mode='lines',
                    name=f'dim {bar["bar_dim"]}',
                    showlegend=showlegendtf,
                    line=dict(color=aptviz_themes.davos_colors[int(bar["bar_dim"])]),
                    hovertemplate =
                        f'<i>id</i>: {bar["bar_id"]}'+
                        f'<br><b>rep</b>: {bar["rep"]}'))



# Add nice effects and such
fig.update_layout(title_text="Barcode")
fig.update_yaxes(title_text = "H_k")
fig.update_xaxes(title_text = "Filtration index")
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.update_layout(hovermode="x unified")
fig.show()

# # fig.write_image("fig1.pdf")
